<a href="https://colab.research.google.com/github/arnaujc91/experiments/blob/main/transformers_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai==2.0.16
!pip install sklearn

In [3]:
!pip install -Uq transformers

     |████████████████████████████████| 1.3MB 5.1MB/s 
     |████████████████████████████████| 890kB 30.8MB/s 
     |████████████████████████████████| 1.1MB 19.3MB/s 
     |████████████████████████████████| 2.9MB 61.8MB/s 


In [2]:
!pip install pytorch-nlp

     |████████████████████████████████| 92kB 3.8MB/s 


In [11]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import torch
import pandas as pd
import re

In [6]:
pretrained_weights = 'gpt2'

tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)



In [14]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [30]:
def rm_first_handle(t):
    return re.sub(r'^@\w* ', '', t)
    
url = 'https://raw.githubusercontent.com/arnaujc91/ULMFit/master/data/Tweets.csv'
tweets = pd.read_csv(url)
tweets = tweets[['airline_sentiment', 'text']] 
tweets['text'] = tweets['text'].map(rm_first_handle)



In [72]:
from torchnlp.encoders.label_encoder import LabelEncoder

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, tweets, text_col, label_col):
        self.tweets = tweets[[text_col]+[label_col]]
        self.encoder = LabelEncoder(tweets[label_col].values)
        # self.tweets['text_tok'] = tweets.loc[:,text_col]
        self.tweets['text_tok'] = self.tweets[text_col].map(TransformersTokenizer(tokenizer).encodes)
        self.tweets.loc[:,'label_enc'] = self.tweets[label_col].map(self.encoder.encode)

    def __getitem__(self, i):
        # if i%2==0:
        #   return torch.tensor([1,2,3,4]), self.tweets.iloc[i]['label_enc']
        # return torch.tensor([1,2,3,4,5,6,7]), self.tweets.iloc[i]['label_enc']
        return self.tweets.iloc[i]['text_tok'], self.tweets.iloc[i]['label_enc']
    
    def __len__(self): 
        return len(self.tweets)

In [73]:
from sklearn.model_selection import train_test_split
t, v = train_test_split(tweets)
train_ds = TweetDataset(t, text_col = 'text', label_col = 'airline_sentiment')
valid_ds = TweetDataset(v, text_col = 'text', label_col = 'airline_sentiment')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [74]:
dls = DataLoaders.from_dsets(train_ds, valid_ds)
dls.cpu()

In [75]:
dls.one_batch()

RuntimeError: ignored

In [70]:
dls.show_batch()

AttributeError: ignored